In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

data = pd.read_csv('./Datasets/IN_youtube_trending_data.csv')

In [2]:
data.head(5)

video_id                                              title  \
0  Iot0eF6EoNA  Sadak 2 | Official Trailer | Sanjay | Pooja | ...   
1  x-KbnJ9fvJc  Kya Baat Aa : Karan Aujla (Official Video) Tan...   
2  KX06ksuS6Xo  Diljit Dosanjh: CLASH (Official) Music Video |...   
3  UsMRgnTcchY  Dil Ko Maine Di Kasam Video | Amaal M Ft.Ariji...   
4  WNSEXJJhKTU  Baarish (Official Video) Payal Dev,Stebin Ben ...   

            publishedAt                 channelId    channelTitle  categoryId  \
0  2020-08-12T04:31:41Z  UCGqvJPRcv7aVFun-eTsatcA    FoxStarHindi          24   
1  2020-08-11T09:00:11Z  UCm9SZAl03Rev9sFwloCdz1g  Rehaan Records          10   
2  2020-08-11T07:30:02Z  UCZRdNleCgW-BGUJf-bbjzQg  Diljit Dosanjh          10   
3  2020-08-10T05:30:49Z  UCq-Fj5jknLsUf-MWSy4_brA        T-Series          10   
4  2020-08-11T05:30:13Z  UCye6Oz0mg46S362LwARGVcA   VYRLOriginals          10   

          trending_date                                               tags  \
0  2020-08-12T00:00:00Z  sadak|sadak 2|mahesh bhatt|vishesh films|pooja...   
1  2020-08-12T00:00:00Z                                             [None]   
2  2020-08-12T00:00:00Z  clash diljit dosanjh|diljit dosanjh|diljit dos...   
3  2020-08-12T00:00:00Z  hindi songs|2020 hindi songs|2020 new songs|t-...   
4  2020-08-12T00:00:00Z  VYRL Original|Mohsin Khan|Shivangi Joshi|Payal...   

   view_count   likes  dislikes  comment_count  \
0     9885899  224925   3979409         350210   
1    11308046  655450     33242         405146   
2     9140911  296533      6179          30058   
3    23564512  743931     84162         136942   
4     6783649  268817      8798          22984   

                                   thumbnail_link  comments_disabled  \
0  https://i.ytimg.com/vi/Iot0eF6EoNA/default.jpg              False   
1  https://i.ytimg.com/vi/x-KbnJ9fvJc/default.jpg              False   
2  https://i.ytimg.com/vi/KX06ksuS6Xo/default.jpg              False   
3  https://i.ytimg.com/vi/UsMRgnTcchY/default.jpg              False   
4  https://i.ytimg.com/vi/WNSEXJJhKTU/default.jpg              False   

   ratings_disabled                                        description  
0             False  Three Streams. Three Stories. One Journey. Sta...  
1             False  Singer/Lyrics: Karan Aujla Feat Tania Music/ D...  
2             False  CLASH official music video performed by DILJIT...  
3             False  Gulshan Kumar and T-Series presents Bhushan Ku...  
4             False  VYRL Originals brings to you ‘Baarish’ - the b...

In [3]:
#drop likes, dislikes, comment_count, thumbnail_link
useless_cols = ['likes', 'dislikes', 'comment_count', 'thumbnail_link','video_id']
modified_data = data.drop(columns = useless_cols)
print(modified_data.shape)

(53327, 11)


In [4]:
#drop description missing rows
modified_data['description'].replace('', np.nan, inplace=True)
modified_data.dropna(subset=['description'], inplace=True)
print(modified_data.shape)

#concatanate text columns
modified_data['channel_text'] = modified_data.title + ' ' + modified_data.channelTitle + ' ' +modified_data.description +'  '+ modified_data.tags
modified_data = modified_data.drop(columns = ['title', 'channelTitle', 'description','tags'])
# modified_data

(52416, 11)


In [8]:
#convert times to timestamps
#change the trending_date format to yyyy-mm-dd from yy.dd.mm
# modified_data['formatted_trending_date'] = pd.to_datetime(modified_data['trending_date'],format="%y.%d.%mT%H:%M:%S.%fZ")
modified_data['formatted_publish_time'] = pd.to_datetime(modified_data['publishedAt'],format="%Y-%m-%dT%H:%M:%S.%fZ")
modified_data ['difference_d'] = (modified_data['formatted_trending_date'] - modified_data['formatted_publish_time']).astype('timedelta64[D]')
modified_data ['difference_h'] = (modified_data['formatted_trending_date'] - modified_data['formatted_publish_time']).astype('timedelta64[h]')


modified_data ['published_year'] = modified_data['formatted_publish_time'].dt.year
modified_data ['published_month'] = modified_data['formatted_publish_time'].dt.month
modified_data ['trending_year'] = modified_data['formatted_trending_date'].dt.year
modified_data ['trending_month'] = modified_data['formatted_trending_date'].dt.month


#drop outliers - past published videos
modified_data = modified_data[modified_data.published_year.isin([2016,2017,2018])]
print(modified_data.shape)

#get small range of view counts
modified_data = modified_data[(modified_data['views'] >= 10000) & (modified_data['views'] <= 50000000)]
print(modified_data.shape)

modified_data['comments_disabled'] = modified_data['comments_disabled'].apply(lambda x: "\"" + str(x) + "\"")
modified_data['ratings_disabled'] = modified_data['ratings_disabled'].apply(lambda x: "\"" + str(x) + "\"")




ValueError: time data '2020-08-12T04:31:41Z' does not match format '%Y-%m-%dT%H:%M:%S.%fZ' (match)

In [ ]:

#drop un-wanted date columns
modified_data = modified_data.drop(columns = ['trending_date', 'publishedAt', 'formatted_trending_date','formatted_publish_time'])
modified_data = modified_data.sample(1200)

modified_data.to_csv('./Datasets/INvideos_Processed_train_1200_with-t_desc.csv',index=False, encoding='utf-8')